<a href="https://colab.research.google.com/github/gmihaila/machine_learning_things/blob/master/learning_tensorflow/tf2_0_text_binary_classification_sentence_embed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Regular Classificaiton [sentence embedding]

In [0]:
!nvidia-smi

Sat Feb  8 22:59:23 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.48.02    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

### Imports

In [0]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
# tf.debugging.set_log_device_placement(True)
import tensorflow_datasets as tfds
import tensorflow_hub as hub
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import time
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

BUFFER_SIZE = 1000
BATCH = 1000
EPOCHS = 20
MAX_LEN = 500

TensorFlow 2.x selected.
Num GPUs Available:  1


### Dataset

In [0]:
train_data, validation_data, test_data = tfds.load(name="imdb_reviews", 
                                                   split=('train[:60%]', 'train[60%:]', 'test'),
                                                   as_supervised=True)

# batch padding
train_batches = (train_data.batch(BATCH))
valid_batched = (validation_data.batch(BATCH))
test_batches = (test_data.batch(BATCH))

# sample
texts, labels = next(iter(train_batches))

print("Text sample:           {}".format(texts[0]))
print("Label:                 {}".format(labels[0]))
print("Encoding batch shape:  {}".format(texts.shape))
print("Labels batch shape:    {}".format(labels.shape))

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete8043KK/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete8043KK/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete8043KK/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.
Text sample:           b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it."
Label:                 0
Encoding batch shape:  (1000,)
Labels batch shape:    (1000,)


### Sequential Model + regular train

In [0]:
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(embedding, input_shape=[], 
                           dtype=tf.string, trainable=True)
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.summary()

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(train_batches,
                    epochs=20,
                    validation_data=valid_batched,
                    verbose=1)

results = model.evaluate(test_data.batch(512), verbose=2)

# loss: 0.320
# accuracy: 0.863
for name, value in zip(model.metrics_names, results):
  print("%s: %.3f" % (name, value))


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 20)                400020    
_________________________________________________________________
dense (Dense)                (None, 16)                336       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
15/15 [==============================] - 6s 379ms/step - loss: 0.7418 - accuracy: 0.5295 - val_loss: 0.7055 - val_accuracy: 0.5644
Epoch 2/20
15/15 [==============================] - 4s 236ms/step - loss: 0.6773 - accuracy: 0.5959 - val_loss: 0.6603 - val_accuracy: 0.6150
Epoch 3/20
15/15 [==============================] - 4s 261ms/step - loss: 0.6365 - accuracy: 0.6

### Sequential Model + gradient tape

Apply with logits if we apply optimizer to values coming form softmax

In [0]:
class Classification(tf.keras.Model):
  def __init__(self, embedding):
    super(Classification, self).__init__()
    self.hub_layer = hub.KerasLayer(embedding, input_shape=[], dtype=tf.string, trainable=True)
    self.l1 = tf.keras.layers.Dense(units=16, activation=tf.nn.relu)
    self.l2 = tf.keras.layers.Dense(units=1, activation=tf.nn.sigmoid)
    return

  def call(self, input_):
    output_ = self.hub_layer(input_)
    output_ = self.l1(output_)
    output_ = self.l2(output_)
    return output_


def loss_func(target, logits):
  bce = tf.losses.BinaryCrossentropy(from_logits=False)
  return bce(y_true=target, y_pred=logits)


@tf.function
def train_step(input_batch, lables_batch, optim):
  loss = 0
  with tf.GradientTape() as tape:
    preds_batch = model(input_batch)
    loss = loss_func(lables_batch, preds_batch)

  variables = model.variables
  gradients = tape.gradient(loss, variables)
  optim.apply_gradients(zip(gradients, variables))
  return loss



model = Classification(embedding="https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1")
model_optimizer = tf.optimizers.Adam()
model_metrics = tf.metrics.Accuracy()

EPOCHS = 20

for epoch in range(EPOCHS):
  batch_loss = 0
  batch_val_loss = 0

  batch_acc = 0
  batch_val_acc = 0

  for batch, (text_batch, label_batch) in enumerate(train_batches, start=1):
    batch_loss += train_step(text_batch, label_batch, model_optimizer)

    preds_batch = model(text_batch)
    preds_batch = tf.keras.backend.round(preds_batch)
    batch_acc += model_metrics(label_batch, preds_batch) 

  batch_loss = tf.get_static_value(batch_loss) / batch
  batch_acc = tf.get_static_value(batch_acc) / batch

  for batch, (text_batch, label_batch) in enumerate(valid_batched, start=1):
    preds_batch = model(text_batch)
    batch_val_loss += loss_func(label_batch, preds_batch)

    preds_batch = tf.keras.backend.round(preds_batch)
    batch_val_acc += model_metrics(label_batch, preds_batch) 

  
  batch_val_loss = tf.get_static_value(batch_val_loss) / batch
  batch_val_acc =tf.get_static_value(batch_val_acc) / batch

  print('Epoch: %s/%s\tloss: %.4f\tacc: %.4f\tval_loss: %.4f\tval_acc: %.4f'%(epoch+1, EPOCHS, batch_loss, batch_acc, batch_val_loss, batch_val_acc))
    


Epoch: 1/20	loss: 0.7262	acc: 0.5716	val_loss: 0.6859	val_acc: 0.5821
Epoch: 2/20	loss: 0.6592	acc: 0.5921	val_loss: 0.6381	val_acc: 0.6026
Epoch: 3/20	loss: 0.6214	acc: 0.6126	val_loss: 0.6104	val_acc: 0.6216
Epoch: 4/20	loss: 0.5954	acc: 0.6301	val_loss: 0.5890	val_acc: 0.6378
Epoch: 5/20	loss: 0.5730	acc: 0.6451	val_loss: 0.5681	val_acc: 0.6516
Epoch: 6/20	loss: 0.5499	acc: 0.6579	val_loss: 0.5465	val_acc: 0.6638
Epoch: 7/20	loss: 0.5270	acc: 0.6694	val_loss: 0.5260	val_acc: 0.6750
Epoch: 8/20	loss: 0.5039	acc: 0.6804	val_loss: 0.5063	val_acc: 0.6855
Epoch: 9/20	loss: 0.4807	acc: 0.6905	val_loss: 0.4875	val_acc: 0.6953
Epoch: 10/20	loss: 0.4583	acc: 0.7001	val_loss: 0.4695	val_acc: 0.7046
Epoch: 11/20	loss: 0.4367	acc: 0.7091	val_loss: 0.4526	val_acc: 0.7133
Epoch: 12/20	loss: 0.4159	acc: 0.7175	val_loss: 0.4369	val_acc: 0.7215
Epoch: 13/20	loss: 0.3960	acc: 0.7254	val_loss: 0.4223	val_acc: 0.7291
Epoch: 14/20	loss: 0.3767	acc: 0.7328	val_loss: 0.4086	val_acc: 0.7364
Epoch: 15/20	lo

### Separate Models + gradient tape

In [0]:
class Embed(tf.keras.Model):
  def __init__(self, embedding):
    super(Embed, self).__init__()
    self.hub_layer = hub.KerasLayer(embedding, input_shape=[], dtype=tf.string, trainable=True)
    return

  def call(self, input_):
    output_ = self.hub_layer(input_)
    return output_

class Dens(tf.keras.Model):
  def __init__(self):
    super(Dens, self).__init__()
    self.l1 = tf.keras.layers.Dense(units=16, activation=tf.nn.relu)
    self.l2 = tf.keras.layers.Dense(units=1, activation=tf.nn.sigmoid)
    return

  def call(self, input_):
    output_ = self.l1(input_)
    output_ = self.l2(output_)
    return output_


def loss_func(target, logits):
  bce = tf.losses.BinaryCrossentropy(from_logits=False)
  return bce(y_true=target, y_pred=logits)


@tf.function
def train_step(input_batch, lables_batch, optim):
  loss = 0
  with tf.GradientTape() as tape:
    preds_batch = model_embed(input_batch)
    preds_batch = model_dens(preds_batch)

    loss = loss_func(lables_batch, preds_batch)

  variables = model_embed.variables + model_dens.variables
  gradients = tape.gradient(loss, variables)
  optim.apply_gradients(zip(gradients, variables))
  return loss



model_embed = Embed(embedding="https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1")
model_dens = Dens()

model_optimizer = tf.optimizers.Adam()
model_metrics = tf.metrics.Accuracy()

EPOCHS = 20

for epoch in range(EPOCHS):
  batch_loss = 0
  batch_val_loss = 0

  batch_acc = 0
  batch_val_acc = 0

  for batch, (text_batch, label_batch) in enumerate(train_batches, start=1):
    batch_loss += train_step(text_batch, label_batch, model_optimizer)

    preds_batch = model_embed(text_batch)
    preds_batch = model_dens(preds_batch)

    preds_batch = tf.keras.backend.round(preds_batch)
    batch_acc += model_metrics(label_batch, preds_batch) 

  batch_loss = tf.get_static_value(batch_loss) / batch
  batch_acc = tf.get_static_value(batch_acc) / batch

  for batch, (text_batch, label_batch) in enumerate(valid_batched, start=1):
    preds_batch = model_embed(text_batch)
    preds_batch = model_dens(preds_batch)

    batch_val_loss += loss_func(label_batch, preds_batch)

    preds_batch = tf.keras.backend.round(preds_batch)
    batch_val_acc += model_metrics(label_batch, preds_batch) 

  
  batch_val_loss = tf.get_static_value(batch_val_loss) / batch
  batch_val_acc =tf.get_static_value(batch_val_acc) / batch

  print('Epoch: %s/%s\tloss: %.4f\tacc: %.4f\tval_loss: %.4f\tval_acc: %.4f'%(epoch+1, EPOCHS, batch_loss, batch_acc, batch_val_loss, batch_val_acc))
    


Epoch: 1/20	loss: 0.6764	acc: 0.5869	val_loss: 0.6475	val_acc: 0.6076
Epoch: 2/20	loss: 0.6281	acc: 0.6194	val_loss: 0.6108	val_acc: 0.6325
Epoch: 3/20	loss: 0.5923	acc: 0.6438	val_loss: 0.5807	val_acc: 0.6538
Epoch: 4/20	loss: 0.5618	acc: 0.6634	val_loss: 0.5543	val_acc: 0.6722
Epoch: 5/20	loss: 0.5339	acc: 0.6803	val_loss: 0.5301	val_acc: 0.6877
Epoch: 6/20	loss: 0.5076	acc: 0.6950	val_loss: 0.5075	val_acc: 0.7015
Epoch: 7/20	loss: 0.4823	acc: 0.7079	val_loss: 0.4862	val_acc: 0.7137
Epoch: 8/20	loss: 0.4577	acc: 0.7194	val_loss: 0.4662	val_acc: 0.7245
Epoch: 9/20	loss: 0.4340	acc: 0.7295	val_loss: 0.4473	val_acc: 0.7341
Epoch: 10/20	loss: 0.4113	acc: 0.7388	val_loss: 0.4296	val_acc: 0.7431
Epoch: 11/20	loss: 0.3892	acc: 0.7474	val_loss: 0.4132	val_acc: 0.7513
Epoch: 12/20	loss: 0.3679	acc: 0.7553	val_loss: 0.3975	val_acc: 0.7590
Epoch: 13/20	loss: 0.3474	acc: 0.7627	val_loss: 0.3836	val_acc: 0.7662
Epoch: 14/20	loss: 0.3283	acc: 0.7697	val_loss: 0.3713	val_acc: 0.7730
Epoch: 15/20	lo